# Anna KaRNNa

In this notebook, we'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [3]:
## TESTS
#print(len(vocab)) --- 83
#print(int_to_vocab)  --- eg. 0:'c'
#print(len(int_to_vocab)) --- 83
#print(vocab_to_int)   --- eg. 'c':0
#print(len(vocab_to_int)) --- 72
print(vocab_to_int['C']) #--- 37

44


Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [4]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see the characters encoded as integers.

In [5]:
encoded[:100]

array([44,  0, 39, 18, 61, 60, 33, 26,  7,  1,  1,  1, 47, 39, 18, 18, 81,
       26, 41, 39, 23,  2, 27,  2, 60, 58, 26, 39, 33, 60, 26, 39, 27, 27,
       26, 39, 27,  2,  8, 60, 80, 26, 60, 69, 60, 33, 81, 26, 59,  6,  0,
       39, 18, 18, 81, 26, 41, 39, 23,  2, 27, 81, 26,  2, 58, 26, 59,  6,
        0, 39, 18, 18, 81, 26,  2,  6, 26,  2, 61, 58, 26, 32, 79,  6,  1,
       79, 39, 81, 25,  1,  1, 65, 69, 60, 33, 81, 61,  0,  2,  6])

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [6]:
len(vocab)

83

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>
We have our text encoded as integers as one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the number of batches we can make from some array `arr`, you divide the length of `arr` by the batch size. Once you know the number of batches and the batch size, you can get the total number of characters to keep.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`n_seqs` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$ where $K$ is the number of batches.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. You'll usually see the first input character used as the last target character, so something like this:
```python
y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
```
where `x` is the input batch and `y` is the target batch.

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

> **Exercise:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [7]:
tx = np.array(encoded[:100])
tx = tx.reshape((10,-1))
tx.shape[1]
print(tx)
ty = np.zeros_like(tx)
ty[:, :-1], ty[:, -1] = tx[:, 1:], tx[:, 0]

[[44  0 39 18 61 60 33 26  7  1]
 [ 1  1 47 39 18 18 81 26 41 39]
 [23  2 27  2 60 58 26 39 33 60]
 [26 39 27 27 26 39 27  2  8 60]
 [80 26 60 69 60 33 81 26 59  6]
 [ 0 39 18 18 81 26 41 39 23  2]
 [27 81 26  2 58 26 59  6  0 39]
 [18 18 81 26  2  6 26  2 61 58]
 [26 32 79  6  1 79 39 81 25  1]
 [ 1 65 69 60 33 81 61  0  2  6]]


In [8]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the batch size and number of batches we can make
    batch_size = n_seqs * n_steps
    n_batches = len(arr) // batch_size
    
    # Keep only enough characters to make full batches
    arr = arr [: n_batches * batch_size]
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr [:,n:n+n_steps ]
        # The targets, shifted by one
        y = np.zeros_like(x)
        #ty[:, :-1], ty[:, -1] = tx[:, 1:], tx[:, 0]
        y[:,:-1], y[:,-1] = x[:,1:], x[:,0] 
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [9]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [10]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[44  0 39 18 61 60 33 26  7  1]
 [26 39 23 26  6 32 61 26 56 32]
 [69  2  6 25  1  1 30 77 60 58]
 [ 6 26 24 59 33  2  6 56 26  0]
 [26  2 61 26  2 58 63 26 58  2]
 [26 46 61 26 79 39 58  1 32  6]
 [ 0 60  6 26 12 32 23 60 26 41]
 [80 26 10 59 61 26  6 32 79 26]
 [61 26  2 58  6 75 61 25 26 62]
 [26 58 39  2 24 26 61 32 26  0]]

y
 [[ 0 39 18 61 60 33 26  7  1  1]
 [39 23 26  6 32 61 26 56 32  2]
 [ 2  6 25  1  1 30 77 60 58 63]
 [26 24 59 33  2  6 56 26  0  2]
 [ 2 61 26  2 58 63 26 58  2 33]
 [46 61 26 79 39 58  1 32  6 27]
 [60  6 26 12 32 23 60 26 41 32]
 [26 10 59 61 26  6 32 79 26 58]
 [26  2 58  6 75 61 25 26 62  0]
 [58 39  2 24 26 61 32 26  0 60]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`. This will be a scalar, that is a 0-D tensor. To make a scalar, you create a placeholder without giving it a size.

> **Exercise:** Create the input placeholders in the function below.

In [11]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
    
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. For example,

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow will create different weight matrices for all `cell` objects. Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

> **Exercise:** Below, implement the `build_lstm` function to create these LSTM cells and the initial state.

In [12]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    # Use a basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell outputs
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob = keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(lstm_size), output_keep_prob = keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character, so we want this layer to have size $C$, the number of classes/characters we have in our text.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$. The output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$. 

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells. We get the LSTM output as a list, `lstm_output`. First we need to concatenate this whole list into one array with [`tf.concat`](https://www.tensorflow.org/api_docs/python/tf/concat). Then, reshape it (with `tf.reshape`) to size $(M * N) \times L$.

One we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

> **Exercise:** Implement the output layer in the function below.

In [13]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        lstm_output: List of output tensors from the LSTM layer
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # Concatenate lstm_output over axis 1 (the columns)
    seq_output = tf.concat(lstm_output, axis=1)
    # Reshape seq_output to a 2D tensor with lstm_size columns
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        # Create the weight and bias variables here
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

>**Exercise:** Implement the loss calculation in the function below.

In [14]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per sequence per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped =  tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [15]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

> **Exercise:** Use the functions you've implemented previously and `tf.nn.dynamic_rnn` to build the network.

In [16]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN with tf.nn.dynamic_rnn 
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state = self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss =  build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

## Hyperparameters

Here are the hyperparameters for the network.

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

In [17]:
batch_size = 10         # Sequences per batch
num_steps = 100          # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001    # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}.ckpt`

> **Exercise:** Set the hyperparameters above to train the network. Watch the training loss, it should be consistently dropping. Also, I highly advise running this on a GPU.

In [18]:
epochs = 5
# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/5...  Training Step: 1...  Training loss: 4.4167...  1.4562 sec/batch
Epoch: 1/5...  Training Step: 2...  Training loss: 4.3206...  1.4905 sec/batch
Epoch: 1/5...  Training Step: 3...  Training loss: 3.8235...  1.6609 sec/batch
Epoch: 1/5...  Training Step: 4...  Training loss: 5.2638...  1.6548 sec/batch
Epoch: 1/5...  Training Step: 5...  Training loss: 4.2985...  1.6026 sec/batch
Epoch: 1/5...  Training Step: 6...  Training loss: 3.8881...  1.6786 sec/batch
Epoch: 1/5...  Training Step: 7...  Training loss: 3.8695...  1.7779 sec/batch
Epoch: 1/5...  Training Step: 8...  Training loss: 3.5358...  1.6801 sec/batch
Epoch: 1/5...  Training Step: 9...  Training loss: 3.5166...  1.7056 sec/batch
Epoch: 1/5...  Training Step: 10...  Training loss: 3.4921...  1.7035 sec/batch
Epoch: 1/5...  Training Step: 11...  Training loss: 3.4714...  1.7978 sec/batch
Epoch: 1/5...  Training Step: 12...  Training loss: 3.4594...  1.8163 sec/batch
Epoch: 1/5...  Training Step: 13...  Training los

Epoch: 1/5...  Training Step: 104...  Training loss: 3.1273...  1.9669 sec/batch
Epoch: 1/5...  Training Step: 105...  Training loss: 3.0883...  1.8360 sec/batch
Epoch: 1/5...  Training Step: 106...  Training loss: 3.1022...  1.8706 sec/batch
Epoch: 1/5...  Training Step: 107...  Training loss: 3.1158...  1.9374 sec/batch
Epoch: 1/5...  Training Step: 108...  Training loss: 3.1339...  1.8009 sec/batch
Epoch: 1/5...  Training Step: 109...  Training loss: 3.0825...  1.8187 sec/batch
Epoch: 1/5...  Training Step: 110...  Training loss: 3.0790...  1.7736 sec/batch
Epoch: 1/5...  Training Step: 111...  Training loss: 3.1293...  1.8110 sec/batch
Epoch: 1/5...  Training Step: 112...  Training loss: 3.0854...  1.8051 sec/batch
Epoch: 1/5...  Training Step: 113...  Training loss: 3.1742...  1.8758 sec/batch
Epoch: 1/5...  Training Step: 114...  Training loss: 3.1485...  1.8881 sec/batch
Epoch: 1/5...  Training Step: 115...  Training loss: 3.0982...  1.8165 sec/batch
Epoch: 1/5...  Training Step

Epoch: 1/5...  Training Step: 206...  Training loss: 2.6647...  1.8167 sec/batch
Epoch: 1/5...  Training Step: 207...  Training loss: 2.6512...  1.8001 sec/batch
Epoch: 1/5...  Training Step: 208...  Training loss: 2.5393...  1.8799 sec/batch
Epoch: 1/5...  Training Step: 209...  Training loss: 2.5650...  1.8250 sec/batch
Epoch: 1/5...  Training Step: 210...  Training loss: 2.5615...  1.9232 sec/batch
Epoch: 1/5...  Training Step: 211...  Training loss: 2.6353...  1.7876 sec/batch
Epoch: 1/5...  Training Step: 212...  Training loss: 2.6206...  1.7793 sec/batch
Epoch: 1/5...  Training Step: 213...  Training loss: 2.5986...  1.8468 sec/batch
Epoch: 1/5...  Training Step: 214...  Training loss: 2.5213...  1.7827 sec/batch
Epoch: 1/5...  Training Step: 215...  Training loss: 2.5511...  1.7901 sec/batch
Epoch: 1/5...  Training Step: 216...  Training loss: 2.5269...  1.8108 sec/batch
Epoch: 1/5...  Training Step: 217...  Training loss: 2.5442...  1.8879 sec/batch
Epoch: 1/5...  Training Step

Epoch: 1/5...  Training Step: 308...  Training loss: 2.3484...  1.8702 sec/batch
Epoch: 1/5...  Training Step: 309...  Training loss: 2.4047...  1.8592 sec/batch
Epoch: 1/5...  Training Step: 310...  Training loss: 2.3248...  1.8725 sec/batch
Epoch: 1/5...  Training Step: 311...  Training loss: 2.3361...  1.8602 sec/batch
Epoch: 1/5...  Training Step: 312...  Training loss: 2.3857...  1.8263 sec/batch
Epoch: 1/5...  Training Step: 313...  Training loss: 2.3404...  1.8336 sec/batch
Epoch: 1/5...  Training Step: 314...  Training loss: 2.3431...  1.7918 sec/batch
Epoch: 1/5...  Training Step: 315...  Training loss: 2.4768...  1.7932 sec/batch
Epoch: 1/5...  Training Step: 316...  Training loss: 2.3484...  1.8310 sec/batch
Epoch: 1/5...  Training Step: 317...  Training loss: 2.4213...  1.8969 sec/batch
Epoch: 1/5...  Training Step: 318...  Training loss: 2.4690...  1.8375 sec/batch
Epoch: 1/5...  Training Step: 319...  Training loss: 2.3698...  1.8066 sec/batch
Epoch: 1/5...  Training Step

Epoch: 1/5...  Training Step: 410...  Training loss: 2.2941...  1.8526 sec/batch
Epoch: 1/5...  Training Step: 411...  Training loss: 2.2587...  1.8350 sec/batch
Epoch: 1/5...  Training Step: 412...  Training loss: 2.2037...  1.8117 sec/batch
Epoch: 1/5...  Training Step: 413...  Training loss: 2.2692...  1.8665 sec/batch
Epoch: 1/5...  Training Step: 414...  Training loss: 2.2678...  1.8277 sec/batch
Epoch: 1/5...  Training Step: 415...  Training loss: 2.2577...  1.8187 sec/batch
Epoch: 1/5...  Training Step: 416...  Training loss: 2.2790...  1.8179 sec/batch
Epoch: 1/5...  Training Step: 417...  Training loss: 2.2702...  1.8799 sec/batch
Epoch: 1/5...  Training Step: 418...  Training loss: 2.2981...  1.7914 sec/batch
Epoch: 1/5...  Training Step: 419...  Training loss: 2.2235...  1.8179 sec/batch
Epoch: 1/5...  Training Step: 420...  Training loss: 2.2731...  1.8137 sec/batch
Epoch: 1/5...  Training Step: 421...  Training loss: 2.2642...  1.8159 sec/batch
Epoch: 1/5...  Training Step

...

Epoch: 5/5...  Training Step: 9521...  Training loss: 1.3618...  1.9159 sec/batch
Epoch: 5/5...  Training Step: 9522...  Training loss: 1.2770...  1.9128 sec/batch
Epoch: 5/5...  Training Step: 9523...  Training loss: 1.1877...  1.9884 sec/batch
Epoch: 5/5...  Training Step: 9524...  Training loss: 1.1398...  2.0510 sec/batch
Epoch: 5/5...  Training Step: 9525...  Training loss: 1.2777...  2.0044 sec/batch
Epoch: 5/5...  Training Step: 9526...  Training loss: 1.2541...  1.9033 sec/batch
Epoch: 5/5...  Training Step: 9527...  Training loss: 1.2319...  1.9674 sec/batch
Epoch: 5/5...  Training Step: 9528...  Training loss: 1.2071...  1.8978 sec/batch
Epoch: 5/5...  Training Step: 9529...  Training loss: 1.2497...  1.9449 sec/batch
Epoch: 5/5...  Training Step: 9530...  Training loss: 1.2637...  1.9904 sec/batch
Epoch: 5/5...  Training Step: 9531...  Training loss: 1.2033...  1.9664 sec/batch
Epoch: 5/5...  Training Step: 9532...  Training loss: 1.2980...  1.9489 sec/batch
Epoch: 5/5...  T

Epoch: 5/5...  Training Step: 9621...  Training loss: 1.2180...  1.9789 sec/batch
Epoch: 5/5...  Training Step: 9622...  Training loss: 1.2232...  1.9579 sec/batch
Epoch: 5/5...  Training Step: 9623...  Training loss: 1.3052...  2.0259 sec/batch
Epoch: 5/5...  Training Step: 9624...  Training loss: 1.3499...  1.9944 sec/batch
Epoch: 5/5...  Training Step: 9625...  Training loss: 1.2818...  2.0269 sec/batch
Epoch: 5/5...  Training Step: 9626...  Training loss: 1.3750...  1.9539 sec/batch
Epoch: 5/5...  Training Step: 9627...  Training loss: 1.3305...  1.9038 sec/batch
Epoch: 5/5...  Training Step: 9628...  Training loss: 1.3007...  1.9934 sec/batch
Epoch: 5/5...  Training Step: 9629...  Training loss: 1.2920...  2.0119 sec/batch
Epoch: 5/5...  Training Step: 9630...  Training loss: 1.1900...  1.9459 sec/batch
Epoch: 5/5...  Training Step: 9631...  Training loss: 1.1874...  1.9949 sec/batch
Epoch: 5/5...  Training Step: 9632...  Training loss: 1.1973...  1.9999 sec/batch
Epoch: 5/5...  T

Epoch: 5/5...  Training Step: 9721...  Training loss: 1.2471...  1.9576 sec/batch
Epoch: 5/5...  Training Step: 9722...  Training loss: 1.2475...  1.9778 sec/batch
Epoch: 5/5...  Training Step: 9723...  Training loss: 1.2494...  2.1456 sec/batch
Epoch: 5/5...  Training Step: 9724...  Training loss: 1.1922...  2.2462 sec/batch
Epoch: 5/5...  Training Step: 9725...  Training loss: 1.2005...  2.1826 sec/batch
Epoch: 5/5...  Training Step: 9726...  Training loss: 1.3809...  2.1461 sec/batch
Epoch: 5/5...  Training Step: 9727...  Training loss: 1.2909...  2.0375 sec/batch
Epoch: 5/5...  Training Step: 9728...  Training loss: 1.1783...  2.0275 sec/batch
Epoch: 5/5...  Training Step: 9729...  Training loss: 1.2470...  2.0400 sec/batch
Epoch: 5/5...  Training Step: 9730...  Training loss: 1.1890...  2.1401 sec/batch
Epoch: 5/5...  Training Step: 9731...  Training loss: 1.2216...  2.1045 sec/batch
Epoch: 5/5...  Training Step: 9732...  Training loss: 1.3158...  2.1671 sec/batch
Epoch: 5/5...  T

Epoch: 5/5...  Training Step: 9821...  Training loss: 1.2640...  1.9584 sec/batch
Epoch: 5/5...  Training Step: 9822...  Training loss: 1.3237...  1.9614 sec/batch
Epoch: 5/5...  Training Step: 9823...  Training loss: 1.2680...  1.9654 sec/batch
Epoch: 5/5...  Training Step: 9824...  Training loss: 1.2793...  2.0645 sec/batch
Epoch: 5/5...  Training Step: 9825...  Training loss: 1.2876...  1.9309 sec/batch
Epoch: 5/5...  Training Step: 9826...  Training loss: 1.2644...  1.9769 sec/batch
Epoch: 5/5...  Training Step: 9827...  Training loss: 1.3857...  2.0405 sec/batch
Epoch: 5/5...  Training Step: 9828...  Training loss: 1.2404...  1.8883 sec/batch
Epoch: 5/5...  Training Step: 9829...  Training loss: 1.3384...  1.8928 sec/batch
Epoch: 5/5...  Training Step: 9830...  Training loss: 1.3901...  1.9549 sec/batch
Epoch: 5/5...  Training Step: 9831...  Training loss: 1.1717...  1.9169 sec/batch
Epoch: 5/5...  Training Step: 9832...  Training loss: 1.2779...  1.8918 sec/batch
Epoch: 5/5...  T

Epoch: 5/5...  Training Step: 9921...  Training loss: 1.3358...  1.8201 sec/batch
Epoch: 5/5...  Training Step: 9922...  Training loss: 1.3362...  1.9240 sec/batch
Epoch: 5/5...  Training Step: 9923...  Training loss: 1.2579...  1.8497 sec/batch
Epoch: 5/5...  Training Step: 9924...  Training loss: 1.3398...  1.8562 sec/batch
Epoch: 5/5...  Training Step: 9925...  Training loss: 1.3304...  1.8209 sec/batch


#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [19]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints\\i9925_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i3000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints\\i3200_l512.ckpt"
all_mo

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [20]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [21]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [22]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints\\i9925_l512.ckpt'

In [23]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints\i9925_l512.ckpt
Farance Levin should not say she and her husband, had such a movement, whom he had not said a striggle of
happy of the carriage the pretty well-buttings was saying as the persons was all the boy of sories of a minute one straight. He felt she would have been directly, when he could the way in walling on him. He was saying.

"I am nor answer. Will
you think to
the mushroom out of that more any one," he said to her and his silence.

"What do you see it to
ask your principle, a cross were already about me all."

"Not a cruak. If the same time with a man interesting
when the mire, and then this was a secrice, we have tried. He's a
little thoughts and so almost sorts
of the balls," answered Levin, like all something or and attractive, but a shirt were she would be to then. "There is stull in the boutting of
the bottle what they were
that I don't want for yourself at the place of the point. I don't suppose I don't say anyt

In [24]:
checkpoint = 'checkpoints/i200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i200_l512.ckpt
Farsde,.

"" hhee on he tat ot are he ood an aot or too hed te or at al aled win ant an an aldd
ate on hees tha tin ase inn ot ti her an he oo tee oo an ione

onn ot e anl har hee in aned tarens th oone otin the
 ter ta io in her toes
tee whe tin int in ha tisin oo ten oong hee ote an thir ion oe hind an teis he aon and the tes ao oid on ite tine ald aind at al and an inned hi on an aode win ao thh tee tin thee wenthin hh en he and ote thee oed
antin hane, ao he the is the sine and ant ho at ha itingg ale ter anedd tit the an aon he etes
inne whe tees tae tone
etinn in ote on hit en to at oe aot an altee his ated, had oid hin tars aonther, ate ante ointh eed theesse aod he intere ase tee int
han tio the ate tens
oint ot toe at on iod he on ans ood wo te ot oe ine at andd ann had hed
ee ito in hhed as heris annd oon at heed tir ine oit itit ithhe
sennt an tat toe ao ane hees oter he thar oon oon, he in tie his on to an

In [25]:
checkpoint = 'checkpoints/i600_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i600_l512.ckpt
Farny and stace on a that was
ig to sit to shat than's to he tore him, the singere thas the camles to
ta deerting of the reoving of the sispiles the loot the thang what shithe sor he dove to the
erand, and to here and alding washer ta to the he stong the teen then she tan wo the her and, what shingted thime was a loult to had his as that it that he tanken as he didene and a lougting the cound to the her was the courd on he seided
andaying to tan it thing of in seat a that,
bound on there worly, and and asded of and te har and his stind of hid bont the
erenter that he chanere a the shid
that
tere than hors to the simleress of thon here whin the canle, and sald and whilk nowe whach, the so she sald the sale who dores and to chentelt this andong wome thay. 
his, and that the cardaded the pistor, and that
thay the candating of hears hore thought of th to shomed he and a lat oulde so canded that his at ale to ta dertell on

In [26]:
checkpoint = 'checkpoints/i1200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1200_l512.ckpt
Farring at a mant, and to a loncter the were of all and he was a mist and theme was neriged, to him alrow and a with wifher there hould to the carrtally.

"She was to them the was a companters and the pestes. That I was nere to so mone of her the some on his was to a mant," sour in a the working with a lowers. "The coult, were were, what she sumple of this his whores, as she was at in thes. I he said, with, she sand all the crouncest was taken, when the where was not,, and had, and was seritult to her have him to be winl hinger.. 
"I a ming thought her his cale, and, and the marr one whore the way to had not the parss," said her the saming on the propritess of her her that was the some
and wifing his aterst with whichers, he
seeded in the parres that here all a the was one as to her seed him the choll of she with the poncess it
her, who have sto got her to there seed to to his the prostly, who
was storded the seare t